In [ ]:
from __future__ import print_function
import caffe
import numpy as np
import os
import sys
import lmdb
from PIL import Image

In [ ]:
def log_progress(sequence, title='', every=1, size=None, end_msg='', removeAtTheEnd=False):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display
    import time
    
    init_time = time.time()

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            elapsed = time.time() - init_time
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    if isinstance(sequence, dict):
                        val = ''
                    else:
                        val=sequence[index-1]
                            
                    timeleft_second = (size-index+1)*float(elapsed)/index
                    hr_left = int(timeleft_second/3600)
                    min_left = int((timeleft_second-3600*hr_left)/60)
                    sec_left = int(timeleft_second-3600*hr_left-60*min_left)
                    label.value = u'{title} : {value} ({index} / {size}) [eta = {hh}h{mm}mn{ss}s]'.format(
                        title=title,
                        value=val,
                        index=index,
                        size=size,
                        hh=hr_left,
                        mm=min_left,
                        ss=sec_left
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = end_msg
        if removeAtTheEnd:
            box.close()
    

In [ ]:
import errno    
import os

def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

In [ ]:
encodings = ['data/labels_segmentation_37d','data/rgb_i_100_8bits','data_spreadout/d_raw_cubehelix_8bits','data_spreadout/d_raw_jet_8bits','data/d_raw_HHA_8bits']

splits=['test','train']

In [ ]:
map_size = 1e12
for encoding in log_progress(encodings, title='Encoding',end_msg='Finished !', removeAtTheEnd=False):
    for split in log_progress(splits, title='Split', removeAtTheEnd=True):
        set_file_path = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data/sets/sunrgbd/'+split+'.txt'
        list_images = open(set_file_path, 'r').read().splitlines()
        pathToLMDB = '/home/jguerry/workspace/dl_data_sunrgbd/LMDB/manual_8bits/'+encoding+'/'+split
        mkdir_p(pathToLMDB)
        db = lmdb.open(pathToLMDB, map_size=map_size)
        with db.begin(write=True) as txn:
            for img_idx in log_progress(list_images, title='Image', removeAtTheEnd=True):
                
                ## DATA LOADING
                img = Image.open('/home/jguerry/workspace/datasets/SUNRGBD_pv/{}/{}.png'.format(encoding,img_idx))
                img = np.array(img, dtype=np.uint8)
                if len(img.shape)==3:
                    img = img[:,:,::-1]
                    #img -= mean_bgr
                    img = img.transpose((2,0,1))
                elif len(img.shape)==2:
                    img = img[np.newaxis, ...]
                else:
                    print("Error : shape not accepted for img.")
            
                datum = caffe.io.array_to_datum(img)
                # On insère le datum et sa clé (idx) dans la LMDB
                #txn.put('{:0>10d}'.format(idx), datum.SerializeToString())
                txn.put(img_idx, datum.SerializeToString())
                
        db.close()
        